In [ ]:
import json, os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
import scipy.stats as stats

In [ ]:
years = ['19-20','20-21','21-22','22-23','23-24']
models = ['DKT', 'SAKT']
fold_nums = range(5)
sample_nums = range(1,11)

In [ ]:
f = open('./Data/dkt_results/cy_DKT_19-20_1.json')
j = json.load(f)
f.close()

In [ ]:
j

In [ ]:
dkt_wy_dict = {}
dkt_cy_dict = {}
for y in years: # DKT
	dkt_wy_dict[y] = []
	for f in fold_nums:
		with open(f'./Data/dkt_results/wy_DKT_{y}_{f}.json') as wyj:
			obj = json.load(wyj)
			dkt_wy_dict[y].append(obj[y][str(f)])
	if y != '23-24':
		dkt_cy_dict[y] = {}
		for s in sample_nums:
			with open(f'./Data/dkt_results/cy_DKT_{y}_{s}.json') as cyj:
				obj = json.load(cyj)
				for test_year, measurement in obj.items():
					if dkt_cy_dict[y].get(test_year) is None:
						dkt_cy_dict[y][test_year] = {}
					dkt_cy_dict[y][test_year][str(s)] = measurement

In [ ]:
sakt_wy_dict = {}
sakt_cy_dict = {}
for y in years: # SAKT
	sakt_wy_dict[y] = []
	for f in fold_nums:
		with open(f'./Data/sakt_results/wy_SAKT_{y}_{f}.json') as wyj:
			obj = json.load(wyj)
			sakt_wy_dict[y].append(obj[y][str(f)])
	if y != '23-24':
		sakt_cy_dict[y] = {}
		for s in sample_nums:
			with open(f'./Data/sakt_results/cy_SAKT_{y}_{s}.json') as cyj:
				obj = json.load(cyj)
				for test_year, measurement in obj.items():
					if sakt_cy_dict[y].get(test_year) is None:
						sakt_cy_dict[y][test_year] = {}
					sakt_cy_dict[y][test_year][str(s)] = measurement

In [ ]:
with open('./Data/formatted_results/cross_year_results_DKT.json','w') as f:
	json.dump(dkt_cy_dict,f)

In [ ]:
with open('./Data/formatted_results/cross_year_results_SAKT.json','w') as f:
	json.dump(sakt_cy_dict,f)

In [ ]:
with open('./Data/formatted_results/within_year_results_DKT.json','w') as f:
	json.dump(dkt_wy_dict,f)

In [ ]:
with open('./Data/formatted_results/within_year_results_SAKT.json','w') as f:
	json.dump(sakt_wy_dict,f)

In [ ]:
st = "./Data/formatted_results/within_year_results_SAKT.json"
st.split('_')[-1].split('.')[0]

In [ ]:
dir = './Data/formatted_results'
mean_dict = {}
res_dict = {}
for fn in os.listdir(dir):
	f = os.path.join(dir,fn)
	model = fn.split('_')[-1].split('.')[0]
	with open(f,'r') as jf:
		obj = json.load(jf)
	if mean_dict.get(model) is None:
		mean_dict[model] = {}
		res_dict[model] = {}
		print(f'initializing {model}')
	if 'cross' in fn:
		# print(mean_dict)
		for train_y, dc in obj.items():
			for test_year, res in dc.items():
				if mean_dict[model].get(test_year) is None:
					# print(f'Initializing {model} {train_y}')
					mean_dict[model][test_year] = {}
					res_dict[model][test_year] = {}
				df = pd.DataFrame(res.values())
				statsd = df.describe()
				mean_dict[model][test_year][train_y] = statsd[0]['mean']
				res_dict[model][test_year][train_y] =list(res.values())
	else:
		# print(mean_dict)
		for year, nums in obj.items():
			if mean_dict[model].get(year) is None:
				# print(f'Initializing {model} {year}')
				res_dict[model][year] = {}
				mean_dict[model][year] = {}
			df = pd.DataFrame(nums)
			statsd = df.describe()
			mean_dict[model][year][year] = statsd[0]['mean']
			res_dict[model][year][year] = nums

In [ ]:
plt.style.available

In [ ]:
plt.style.use('seaborn-v0_8-paper')
for model, model_dct in res_dict.items():
	for eval_year, dct in model_dct.items():
		train_years = dct.keys()
		aucs = dct.values()
		plt.boxplot(aucs,tick_labels=train_years)
		plt.xlabel('Model Training Year')
		plt.ylabel('AUC measurements')
		plt.title(f'{model} evals for {eval_year}')
		plt.show()

# Statistical testing methodology:
- Run Shapiro-Wilk test on auc data from each test-train pair
- Run Anova on eval years (within model)
- Run Anova on eval years (across models?)

In [ ]:
num_nonnormal = 0
num_heteroscedastic = 0
total_num = 0
for model, mod_d in res_dict.items():
	for eval_year, year_d in mod_d.items():
		variances = []
		for train_year, nums in year_d.items():
			test_result = stats.shapiro(nums)
			variances.append(stats.describe(nums).variance)
			total_num += 1
			if test_result.pvalue <= 0.05:
				print(f"Normality assumption violated for model {model} trained on {train_year} evaluated on {eval_year}")
				print(f'p={test_result.pvalue}')
				# stats.probplot(nums,dist='norm', plot=plt)
				# plt.show()
				num_nonnormal += 1
		if max(variances) / min(variances) >= 2:
			num_heteroscedastic += 1
			print(f"Heteroscedasticity detected for {model} eval on {eval_year}")
			print(f'Ratio: {max(variances) / min(variances)}')

In [ ]:
num_nonnormal / total_num

In [ ]:
num_heteroscedastic / total_num

## Shapiro Wilk results
 Roughly 18% of samples are non-normal w/ alpha of 0.1, down to 11% of samples w/alpha of 0.05. I think using t-tests/ANOVAs to compare samples is warranted.
## Heteroscedasticity results
 ~23% of samples showed heteroscedasticity,

In [ ]:
# Model Significance Tests
model_test_results = {}
for model, model_dct in res_dict.items():
	model_test_results[model] = {}
	for eval_year, year_d in model_dct.items():
		if len(year_d.keys()) == 1: # Can't really do anything with only one sample
			pass
		elif len(year_d.keys()) == 2: # t-test
			vals = list(year_d.values())
			model_test_results[model][eval_year] = stats.ttest_ind(vals[0],vals[1],equal_var=False)
		elif len(year_d.keys()) == 3: # ANOVA
			vals = list(year_d.values())
			model_test_results[model][eval_year] = stats.f_oneway(vals[0],vals[1],vals[2])
		elif len(year_d.keys()) == 4: # ANOVA
			vals = list(year_d.values())
			model_test_results[model][eval_year] = stats.f_oneway(vals[0],vals[1],vals[2],vals[3])
		elif len(year_d.keys()) == 5: # ANOVA
			vals = list(year_d.values())
			model_test_results[model][eval_year] = stats.f_oneway(vals[0],vals[1],vals[2],vals[3],vals[4])

In [ ]:
model_test_results

In [ ]:
model_test_results['SAKT']

# Overall rot analysis

In [ ]:
biglist = []
for model, mod_dict in res_dict.items():
	for eval_year, year_dict in mod_dict.items():
		for train_year, nums in year_dict.items():
			for i in nums:
				biglist.append([model,eval_year,train_year,i])

In [ ]:
df = pd.DataFrame(biglist,columns=['model','eval_year','train_year','auc'])

In [ ]:
import seaborn as sns
eval_years = df.eval_year.unique()

for y in eval_years:
	subset = df[df['eval_year'] == y]
	if y != '19-20':
		sns.barplot(x=subset['model'],y=subset['auc'],hue=subset['train_year'])
		plt.legend(loc='lower left')
	else:
		sns.barplot(x=subset['model'],y=subset['auc'])
	plt.xlabel('KT Model',fontsize=18)
	plt.ylabel('AUC measurements',fontsize=18)
	plt.title(f'Models evaluated on Academic Year {y}',fontsize=20)
	
	plt.show()

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
model = ols("auc ~ C(model) + C(eval_year) + C(train_year) + C(model):C(eval_year):C(train_year)", data=df).fit(cov_type='hc3')
sm.stats.anova_lm(model, typ=2, robust='hc3')

In [ ]:
model = ols("auc ~ C(model) + C(eval_year) + C(train_year) + C(model):C(eval_year) +C(train_year):C(eval_year) + C(model):C(train_year)", data=df).fit(cov_type='hc3')
sm.stats.anova_lm(model, typ=2, robust='hc3')

In [ ]:
model = ols("auc ~ C(model) * C(eval_year) * C(train_year)", data=df).fit(cov_type='hc3')
sm.stats.anova_lm(model, typ=2, robust='hc3')

In [ ]:
model = ols("auc ~ C(model) + C(eval_year) + C(train_year) + C(model):C(eval_year) +C(train_year):C(eval_year) + C(model):C(train_year):C(eval_year)", data=df).fit(cov_type='hc3')
sm.stats.anova_lm(model, typ=2, robust='hc3')